<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#스키마" data-toc-modified-id="스키마-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>스키마</a></span></li><li><span><a href="#칼럼과-표현식" data-toc-modified-id="칼럼과-표현식-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>칼럼과 표현식</a></span><ul class="toc-item"><li><span><a href="#칼럼" data-toc-modified-id="칼럼-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>칼럼</a></span></li><li><span><a href="#표현식" data-toc-modified-id="표현식-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>표현식</a></span></li><li><span><a href="#레코드와-로우" data-toc-modified-id="레코드와-로우-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>레코드와 로우</a></span></li></ul></li><li><span><a href="#DataFrame의-트랜스포메이션" data-toc-modified-id="DataFrame의-트랜스포메이션-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>DataFrame의 트랜스포메이션</a></span><ul class="toc-item"><li><span><a href="#데이터프레임-생성" data-toc-modified-id="데이터프레임-생성-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>데이터프레임 생성</a></span></li><li><span><a href="#트랜스포메이션-종류" data-toc-modified-id="트랜스포메이션-종류-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>트랜스포메이션 종류</a></span><ul class="toc-item"><li><span><a href="#select-&amp;-selectExpr" data-toc-modified-id="select-&amp;-selectExpr-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span><code>select</code> &amp; <code>selectExpr</code></a></span></li></ul></li><li><span><a href="#스파크-데이터-타입으로-변환하기" data-toc-modified-id="스파크-데이터-타입으로-변환하기-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>스파크 데이터 타입으로 변환하기</a></span></li><li><span><a href="#칼럼-추가하기" data-toc-modified-id="칼럼-추가하기-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>칼럼 추가하기</a></span></li><li><span><a href="#칼럼명-변경하기" data-toc-modified-id="칼럼명-변경하기-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>칼럼명 변경하기</a></span><ul class="toc-item"><li><span><a href="#예약-문자와-키워드" data-toc-modified-id="예약-문자와-키워드-3.5.1"><span class="toc-item-num">3.5.1&nbsp;&nbsp;</span>예약 문자와 키워드</a></span></li></ul></li><li><span><a href="#대소문자-구분" data-toc-modified-id="대소문자-구분-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>대소문자 구분</a></span></li><li><span><a href="#칼럼-제거하기" data-toc-modified-id="칼럼-제거하기-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>칼럼 제거하기</a></span></li><li><span><a href="#칼럼의-데이터-타입-변경하기" data-toc-modified-id="칼럼의-데이터-타입-변경하기-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>칼럼의 데이터 타입 변경하기</a></span></li><li><span><a href="#로우-필터링하기" data-toc-modified-id="로우-필터링하기-3.9"><span class="toc-item-num">3.9&nbsp;&nbsp;</span>로우 필터링하기</a></span></li><li><span><a href="#고유한(unique)-로우-얻기" data-toc-modified-id="고유한(unique)-로우-얻기-3.10"><span class="toc-item-num">3.10&nbsp;&nbsp;</span>고유한(unique) 로우 얻기</a></span></li><li><span><a href="#무작위-샘플-만들기" data-toc-modified-id="무작위-샘플-만들기-3.11"><span class="toc-item-num">3.11&nbsp;&nbsp;</span>무작위 샘플 만들기</a></span></li><li><span><a href="#임의-분할하기" data-toc-modified-id="임의-분할하기-3.12"><span class="toc-item-num">3.12&nbsp;&nbsp;</span>임의 분할하기</a></span></li><li><span><a href="#로우-합치기와-추가하기" data-toc-modified-id="로우-합치기와-추가하기-3.13"><span class="toc-item-num">3.13&nbsp;&nbsp;</span>로우 합치기와 추가하기</a></span></li><li><span><a href="#로우--정렬하기" data-toc-modified-id="로우--정렬하기-3.14"><span class="toc-item-num">3.14&nbsp;&nbsp;</span>로우  정렬하기</a></span></li><li><span><a href="#로우-수-제한하기" data-toc-modified-id="로우-수-제한하기-3.15"><span class="toc-item-num">3.15&nbsp;&nbsp;</span>로우 수 제한하기</a></span></li><li><span><a href="#repartition-&amp;-coalesce" data-toc-modified-id="repartition-&amp;-coalesce-3.16"><span class="toc-item-num">3.16&nbsp;&nbsp;</span>repartition &amp; coalesce</a></span></li><li><span><a href="#드라이버로-로우-데이터-수집" data-toc-modified-id="드라이버로-로우-데이터-수집-3.17"><span class="toc-item-num">3.17&nbsp;&nbsp;</span>드라이버로 로우 데이터 수집</a></span></li></ul></li></ul></div>

In [1]:
# 데이터 로드
df = spark.read.format('json').load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json')

In [2]:
# 스키마 살펴보기
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



# 스키마

In [3]:
spark.read.format('json').load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json').schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

- 스키마를 출력한 결과
    * ``StructType``이라는 리스트 안에 각 칼럼별로 ``StructField``가 담겨져 있음
    * 즉, 스키마 = 여러개의 ``StructField`` 타입 필드로 구성된 ``StructType`` 객체
- 데이터 타입과 스키마의 데이터 타입이 불일치하면 스파크에서는 오류 발생
- 스파크에서는 자체 데이터 타입을 사용하므로 사용하는 언어 API(ex.Python, R 등)의 데이터 타입을 사용할 수 없음

- 스키마 정의시 **메타 데이터**를 정의할 수도 있음
    * **메타 데이터란, 해당 칼럼과 관련된 정보이며 추후 스파크의 머신러닝 라이브러리에서 사용함**

In [4]:
# 데이터프레임에 스키마를 만들고 적용하는 예제
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), True),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
    StructField('count', LongType(), False, metadata={'hello': 'world'})
])

# 데이터 로드 시 스키마 정의
df = spark.read.format('json').schema(myManualSchema)\
     .load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json')

# 칼럼과 표현식
- 표현식으로 데이터프레임의 칼럼을 선택, 조작, 제거 가능
- 스파크의 칼럼은 표현식을 사용해 레코드(Row) 단위로 계산한 값을 단순하게 나타내는 논리적인 구조
- 칼럼의 실제값을 얻으려면 레코드(Row)가 필요하고 레코드를 얻으려면 데이터프레임이 필요

## 칼럼

- 칼럼을 생성, 참조할 수 있는 방법은 여러가지가 있지만 ``col()``이나 ``column()``을 사용하는 것이 가장 간단. 소괄호 인자에는 ``칼럼명``을 입력

In [5]:
from pyspark.sql.functions import col, column

col('someColumnName')
column('someColumnName')

Column<b'someColumnName'>

- 이 때 해당 칼럼이 로드한 DataFrame에 있는지 여부는 알 수 없음. 알려면 **카탈로그**에 저장된 정보와 비교해야 함. 그런데 이 비교하는 단계는 **분석기**가 트랜스포메이션으로 구성된 논리적실행계획을 검증 전/후로 가는 단계에 걸쳐있음

- 명시적 컬럼 참조. ``col()`` 사용. 이는 데이터프레임 조인 시 특정 칼럼을 참조하는 데 사용
- 단, ``col()``을 사용해 명시적으로 컬럼을 정의하면 **분석기** 실행 단계에서 컬럼 확인 절차를 생략함!(그러므로 더 빨라지겠지..?)
- Pyspark에서는 ``df['column_name']``을 사용

In [6]:
df['count']

Column<b'count'>

## 표현식
- 표현식이란, 데이터프레임의 레코드의 여러값에 대한 **트랜스포메이션의 집합**
- ``expr()``과 ``col()``로 특정 칼럼을 참조하는 것은 동일
- 예를 들어, ``expr('SomeCol - 5')`` == ``col('someCol') - 5`` == ``expr(SomeCol') - 5``

In [7]:
from pyspark.sql.functions import expr

expr("(((SomeCol + 5) * 200) - 6) < otherCol")

Column<b'((((SomeCol + 5) * 200) - 6) < otherCol)'>

- 위 데이터프레임 코드는 SQL의 SELECT 구문으로 해당 표현식을 동일하게 사용해도 동일한 결과를 생성. 왜냐하면 실행 시점에 동일한 논리 트리로 컴파일되기 때문. 성능도 동일

In [8]:
# 데이터프레임의 컬럼에 접근해보기
spark.read.format('json').load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json')\
     .columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## 레코드와 로우

In [9]:
# 가장 첫 번째 레코드(Row 객체)를 반환
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

- Row객체인 레코드를 생성해보자
- 단, Row는 스키마를 갖고 있지 않고 데이터프레임만 스키마를 가짐. 따라서 Row 생성 시, 데이터프레임의 명시된 스키마 순서와 동일하게 해주어야 함

In [15]:
from pyspark.sql import Row

myRow = Row('Hello', None, 1, True) # 마지막 False는 nullable에 대한 인자!?

In [16]:
# 로우의 데이터에 접근하기
print(myRow[0], myRow[1])

Hello None


# DataFrame의 트랜스포메이션
## 데이터프레임 생성
- 기본 트렌스포메이션 결과를 확인하기 위해 임시 뷰로 테이블 등록

In [17]:
df = spark.read.format('json').load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json')
df.createOrReplaceTempView('dfTable')

In [18]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [19]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, LongType

myManualSchema = StructType([
    StructField('some', StringType(), True),
    StructField('col', StringType(), True),
    StructField('names', LongType(), False)
])

myRow = Row("Hello", None, 1) # 데이터프레임이 갖는 Schema 순서로 정의
myDf = spark.createDataFrame([myRow], schema=myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



## 트랜스포메이션 종류
- 칼럼이나 표현식을 사용하는 ``select``
- 문자열 표현식을 사용하는 ``selectExpr``
- 매서드로 사용할 수 없는 ``org.apache.spark.sql.functions`` 패키지에 포함된 다양한 함수
<br><br>
- 위 3가지의 방법으로 데이터프레임을 다룰 때 필요한 대부분의 트랜스포메이션을 수행할 수 있음
### ``select`` & ``selectExpr``
- 데이터 테이블에 SQL 쿼리문을 실행하는 것처럼 사용 가능

In [20]:
df.select('DEST_COUNTRY_NAME').show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [21]:
# 여러개의 칼럼 가져오기
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [22]:
# 다음과 같이 다양한 방법들로 칼럼을 참조할 수 있음
from pyspark.sql.functions import expr, col, column

df.select(expr("DEST_COUNTRY_NAME"),
         col("DEST_COUNTRY_NAME"),
         column("DEST_COUNTRY_NAME")).show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [23]:
# 위와 같이 col(expr/column) 객체와 단순한 문자열을 함께 섞어써도 에러 발생하지 않음(책에선 에러가 발생한다고 했으나...)
df.select(col("DEST_COUNTRY_NAME"), "DEST_COUNTRY_NAME").show(5)

+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+
|    United States|    United States|
|    United States|    United States|
|    United States|    United States|
|            Egypt|            Egypt|
|    United States|    United States|
+-----------------+-----------------+
only showing top 5 rows



- 위 방법 중 ``expr``은 가장 유연하게 칼럼을 참조하는 방법
    - ``AS(일종의 alias)``를 사용해 칼럼 이름을 재정의 가능
    - ``alias``한 것을 다시 ``alias()`` 메소드로 원래대로 변경 가능

In [24]:
df.select(expr("DEST_COUNTRY_NAME AS Destination")).show(2)

+-------------+
|  Destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [25]:
df.select(expr("DEST_COUNTRY_NAME AS Destination").alias("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



- 위와 같이 ``select()``안에 ``expr()``을 같이 사용하는 것을 효율적이고 간단하게 할 수 있는 것이 ``selectExpr``

In [26]:
df.selectExpr("DEST_COUNTRY_NAME AS Destination", "count").show(3)

+-------------+-----+
|  Destination|count|
+-------------+-----+
|United States|   15|
|United States|    1|
|United States|  344|
+-------------+-----+
only showing top 3 rows



- ``selectExpr``은 새로운 데이터프레임을 생성하는(ex.새로운 칼럼을 만들어내는 트렌스포메이션을 지정하면서 새로운 데이터프레임이 생성됨) 복잡한 표현식을 간단하게 만드는 도구

In [27]:
# 출발지와 도착지가 같은지 나타내는 새로운 칼럼 만들기
df.selectExpr("*", "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) AS withinCountry").show(4)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
|            Egypt|      United States|   15|        false|
+-----------------+-------------------+-----+-------------+
only showing top 4 rows



In [28]:
# 간단한 집계(평균, 최댓값, 최소값 등) 가능
df.selectExpr("AVG(count)", "COUNT(DEST_COUNTRY_NAME)").show(5)

+-----------+------------------------+
| avg(count)|count(DEST_COUNTRY_NAME)|
+-----------+------------------------+
|1770.765625|                     256|
+-----------+------------------------+



## 스파크 데이터 타입으로 변환하기
- 명시적인 값(상숫값 또는 추후 비교에 사용할 Scala값 같은 것들)을 스파크에 전달해주기 위해 스파크 데이터 타입으로 변환해야 함
- 이 때, 리터럴(``lit()``)을 사용함. 리터럴은 스파크가 이해할 수 있는 값(스파크 데이터 타입)으로 변환함

In [29]:
from pyspark.sql.functions import lit

df.select(col("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



## 칼럼 추가하기
- DataFrame의 ``withColumn("새로운칼럼이름", 표현식)`` 메서드를 사용

In [30]:
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [31]:
# 출발지와 도착지가 같은지 여부를 불리언 타입으로 하는 새로운 칼럼 만들기
df.withColumn("Binary", expr("DEST_COUNTRY_NAME == ORIGIN_COUNTRY_NAME")).show(5)

+-----------------+-------------------+-----+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Binary|
+-----------------+-------------------+-----+------+
|    United States|            Romania|   15| false|
|    United States|            Croatia|    1| false|
|    United States|            Ireland|  344| false|
|            Egypt|      United States|   15| false|
|    United States|              India|   62| false|
+-----------------+-------------------+-----+------+
only showing top 5 rows



In [35]:
# col 사용시 에러 발생
df.withColumn("binary", col("DEST_COUNTRY_NAME == ORIGIN_COUNTRY_NAME")).show(5)

AnalysisException: cannot resolve '`DEST_COUNTRY_NAME == ORIGIN_COUNTRY_NAME`' given input columns: [DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count];;
'Project [DEST_COUNTRY_NAME#55, ORIGIN_COUNTRY_NAME#56, count#57L, 'DEST_COUNTRY_NAME == ORIGIN_COUNTRY_NAME AS binary#314]
+- Relation[DEST_COUNTRY_NAME#55,ORIGIN_COUNTRY_NAME#56,count#57L] json


- ``withColumn``은 칼럼명을 rename할 수도 있음. 그러나 어떻게 보면 동일한 칼럼이 중복적으로 발생하게 됨

In [37]:
df.withColumn("DEST_COUNTRY_NAME2", col("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'DEST_COUNTRY_NAME2']

## 칼럼명 변경하기
- ``withColumnRenamed("원래칼럼명", "새로운칼럼명")``로 변경 가능

In [38]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "Destination").show(2)

+-------------+-------------------+-----+
|  Destination|ORIGIN_COUNTRY_NAME|count|
+-------------+-------------------+-----+
|United States|            Romania|   15|
|United States|            Croatia|    1|
+-------------+-------------------+-----+
only showing top 2 rows



### 예약 문자와 키워드
- 공백이나 하이픈(-)같은 예약 문자는 칼럼명에 포함시킬 수 없음. 만약 사용하려면 백틱( ` )을 사용해 이스케이핑해야 함

In [39]:
# 이스케이핑이 필요하지 않은 경우 -> withColumn 첫 번째 인자에 새로운 칼럼명을 넣을 때
dfWithLongColName = df.withColumn("This Long Column-Name", expr("ORIGIN_COUNTRY_NAME"))
dfWithLongColName.show(2)

+-----------------+-------------------+-----+---------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|This Long Column-Name|
+-----------------+-------------------+-----+---------------------+
|    United States|            Romania|   15|              Romania|
|    United States|            Croatia|    1|              Croatia|
+-----------------+-------------------+-----+---------------------+
only showing top 2 rows



In [41]:
# 이스케이핑이 필요한 경우 -> 참조하는 표현식에 예약 문자가 있기 때문에 백틱 사용
dfWithLongColName.selectExpr("`This Long Column-Name`",
                            "`This Long Column-Name` as `New Col`").show(2)

+---------------------+-------+
|This Long Column-Name|New Col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



In [42]:
# 조회할 때도 칼럼 명에 예악문자나 키워드가 포함되어 있다면 백틱 추가해서 조회!
dfWithLongColName.select(expr("`This Long Column-Name`"), col("`This Long Column-Name`")).show(2)

+---------------------+---------------------+
|This Long Column-Name|This Long Column-Name|
+---------------------+---------------------+
|              Romania|              Romania|
|              Croatia|              Croatia|
+---------------------+---------------------+
only showing top 2 rows



## 대소문자 구분
- 기본적으로 스파크는 대소문자를 구분하지 않음. 하지만 지정해서 대소문자를 구분하게 할 수 있음

In [43]:
# pyspark
spark.conf.set('spark.sql.caseSensitive', True)

## 칼럼 제거하기
- ``select``로 제거(필요없는 칼럼영 빼고 조회)할 수도 있지만 ``drop``으로도 가능
- ``drop``인자에 여러개의 칼럼명을 넣어 다수의 칼럼들을 제거 가능

In [44]:
# drop
df.drop("ORIGIN_COUNTRY_NAME").columns

['DEST_COUNTRY_NAME', 'count']

In [45]:
# drop multiple columns
dfWithLongColName.drop("count", "This Long Column-Name").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



## 칼럼의 데이터 타입 변경하기
- ``cast``메서드 사용

In [46]:
# int형 -> string으로
ex_df = df.withColumn("count2", col("count").cast("string"))
ex_df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- count2: string (nullable = true)



## 로우 필터링하기
- True/False를 판별하는 표현식을 만들어 False인 로우들을 걸러내기
- ``where``과 ``filter`` 사용(두 방법 모두 같은 연산을 수행하며 같은 파라미터 타입을 사용)

In [47]:
# where
df.where("count < 2").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
|            Malta|      United States|    1|
|    United States|          Gibraltar|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [48]:
# filter
df.filter("count < 2").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
|            Malta|      United States|    1|
|    United States|          Gibraltar|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



- 필터링 조건을 여러개 붙일 수 있음. 그러나 스파크는 모든 필터링 작업을 동시에 수행하기 때문에 필터를 여러개 지정하되 필터링 순서는 스파크의 판단에 맡겨야 함
    * 그러므로 여러개의 조건들 사이에 시간 순서의 종속성이 존재하면 안 되며 조건 끼리는 독립성이 존재해야 함

In [49]:
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



## 고유한(unique) 로우 얻기
- ``distinct`` 메서드 사용

In [50]:
# 출발지, 도착지가 고유한 데이터
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

256

- 출발지 -> 도착지가 유니크한 값들이 256개나 있음

In [51]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

## 무작위 샘플 만들기
- 데이터프레임의 ``sample`` 메서드 사용
- 표본 데이터 추출 비율을 지정할 수 있으며 복원추출 또는 비복원 추출을 지정할 수 있음

In [52]:
seed = 5
withReplacement = False
fraction = 0.5

df.sample(withReplacement=withReplacement, fraction=fraction, seed=seed).count()

138

## 임의 분할하기
- 보통 Train/Test 데이터셋 분할시 자주 사용
- ``randomSplit`` 사용

In [53]:
dataFrames = df.randomSplit([0.25, 0.75], seed=42)
print(dataFrames,'\n', type(dataFrames))

[DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint], DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]] 
 <class 'list'>


In [54]:
print(dataFrames[0].count(), dataFrames[1].count())
dataFrames[0].count() > dataFrames[1].count()

63 193


False

## 로우 합치기와 추가하기
- 데이터프레임은 불변성이기 때문에 로우를 추가하려면 추가할 로우를 새로운 데이터프레임으로 만들고 기존의 데이터프레임과 결합(``union``)하는 방향으로 가야 함
- 결합할 시, 반드시 기존의 데이터프레임의 스키마와 칼럼 수가 동일해야 함!
- 참고로 ``union``은 현재 스키마가 아닌 칼럼 위치를 기반으로 동작하기 때문에 사용자가 생각한 대로 칼렴들이 자동 정렬되어 있지 않을 수 있음!

In [58]:
# 새로운 데이터프레임 만들기 위해 Row 객체를 만들어서 RDD 만들어야 함
from pyspark.sql import Row

schema = df.schema  # 기존의 데이터프레임의 스키마

# 책 속 'L'은 Long Integer Type을 따로 정의해야 하는 Python 2.X 버전이라 그럼
newRows = [Row('New Country', 'Other Country', 5),
          Row('New Country 2', 'Other Country 3', 1)]
parallelizeRows = spark.sparkContext.parallelize(newRows)  # RDD로 변환
newDF = spark.createDataFrame(parallelizeRows, schema=schema)  # DF으로 변환

# 기존DF.union(새로운DF) ~ 
df.union(newDF)\
  .where('count = 1')\
  .where(col('ORIGIN_COUNTRY_NAME') != 'United States')\
  .where(col('DEST_COUNTRY_NAME') != 'United States')\
  .show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



## 로우  정렬하기
- ``sort``, ``orderBy`` 메서드를 사용
- ``orderBy`` 내부 코드에서 ``sort``를 사용하기 때문에 두 메소드 모두 동일한 방식으로 동작
- 디폴트는 오름차순(ASC) 정렬

In [59]:
df.sort('count').show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [60]:
# 여러가지 정렬 기준 가능
df.orderBy('count', 'DEST_COUNTRY_NAME').show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [61]:
# col() 메소드를 사용해서도 정렬 기준 제시 가능
df.orderBy(col('count'), col('DEST_COUNTRY_NAME')).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



- 정렬 기준을 명확히 지정하려면 ``asc``, ``desc`` 메소드를 임포트해서 사용

In [69]:
from pyspark.sql.functions import asc, desc

# expr 메소드를 통해서 표현식으로 정렬 기준 제시
df.orderBy(expr("count").desc()).show(5)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+
only showing top 5 rows



- ``df.orderBy(expr("count desc")).show(5)``
- 위와 같이 표현하면 ``desc``가 ``count``의 alias로 간주함. 따라서 정렬이 안됨. 그러므로 아래의 방법들을 사용하기
    * 관련 <a href='https://stackoverflow.com/questions/53772271/apache-spark-2-0-expression-string-to-orderby-sort-column-in-descending-o'>stackoverflow</a>

In [70]:
df.orderBy('count', ascending=False).show(5)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+
only showing top 5 rows



In [71]:
df.orderBy(col('count').desc(), col('DEST_COUNTRY_NAME').asc()).show(5)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+
only showing top 5 rows



- ``asc_nulls_first``, ``desc_nulls_first``, ``asc_nulls_last``, ``desc_nulls_last``로 DF에서 ``Null``값이 표시되는 기준을 지정할 수 있음(``Null``값이 처음/마지막에 오도록 설정 가능)
<br><br>
- 또한 트랜스포메이션을 처리하기 이전에 성능을 최적화하기 위해 **파티션별 정렬을 수행**하기도 함. 해당 정렬은 ``sortWithinPartitions``메서드로 가능

In [72]:
spark.read.format('json').load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json')\
     .sortWithinPartitions('count')

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

## 로우 수 제한하기
- ``limit`` 메서드 사용

In [73]:
df.orderBy(col('count'), ascending=False).limit(5).show()

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+



## repartition & coalesce
- 물리적 데이터 구성을 제어함으로써 최적화 기법
- ``repartition`` 호출 시 전체 데이터를 셔플
    * 향후 사용할 파티션 수 > 현재 파티션 수
    * 칼럼을 기준으로 파티션을 만드는 경우
    * 위 2가지의 경우에만 사용
    
- 자주 필터링하는 칼럼을 기준으로 파티션을 재분배 즉, 데이터를 재분할

In [74]:
df.rdd.getNumPartitions() # 현재 파티션 1개

1

In [75]:
# 파티션 5개로 데이터를 재분할
test = df.repartition(5)
test.rdd.getNumPartitions()

5

In [76]:
# 자주사용하는 칼럼을 기준으로 파티션 재분할
test2 = df.repartition(col('DEST_COUNTRY_NAME'))
test2.rdd.getNumPartitions()

200

In [77]:
# 자주 사용하는 칼럼과 재분할 시킬 파티션 개수 정의 가능
test3 = df.repartition(5, col('DEST_COUNTRY_NAME'))
test3.rdd.getNumPartitions()

5

- ``coalesce`` 메서드는 파티션을 **줄일 때!** 사용! 즉, 데이터에 셔플을 가하지 않고 파티션을 병합하기 위함!


In [78]:
# 파티션을 5개로 분할시키고 다시 2개의 파티션으로 병합하는 경우
test4 = df.repartition(5, col('DEST_COUNTRY_NAME')).coalesce(2)
test4.rdd.getNumPartitions()

2

## 드라이버로 로우 데이터 수집
- 스파크는 드라이버에서 클러스터 상태 정보를 갖고 있음
- **로컬 환경에서 데이터를 다루려면 드라이버로 데이터를 수집**해야 함!(아직, 드라이버로 데이터를 수집하는 연산은 안배움)
- 연산 중 ``collect``는 전체 DF의 모든 데이터를 수집하며 ``take``는 상위 N개의 로우를 반환, ``show``는 여러 로우를 보기 좋게 출력

In [81]:
collectDF = df.limit(10)
collectDF.take(5) # 상위 5개의 로우를 반환
collectDF.show(5, truncate=True)
collectDF.collect() # 전체 DF의 모든 데이터를 수집


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

- 전체 데이터셋에 대한 반복(iteration)처리를 위해 드라이버로 로우를 모으는 또 다른 방법: ``toLocalIterator``
    * 해당 메서드는 이터레이터로 모든 파티션의 데이터를 드라이버에 전달
    * 해당 메서드로 데이터셋의 파티션을 차례로 하나씩 반복처리 가능

In [82]:
collectDF.toLocalIterator() # generator

<generator object _local_iterator_from_socket.<locals>.PyLocalIterable.__iter__ at 0x7fad4f858950>

In [84]:
for i in collectDF.toLocalIterator():
    print(i)

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1)
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)
Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15)
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1)
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62)
Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588)
Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40)
Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)


In [86]:
print(next(collectDF.toLocalIterator()))
print(next(collectDF.toLocalIterator()))
print(next(collectDF.toLocalIterator()))
print(next(collectDF.toLocalIterator()))

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)
